In [1]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics import classification_report

#### Loading the Data

In [2]:
df = pd.read_csv("Bank_Personal_Loan_Modelling.csv")

In [3]:
print("Shape : ", df.shape)
df.head()

Shape :  (5000, 14)


,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


#### Data Cleaning and Feature Engineering

In [4]:
# Deleting Columns which are not necessary
df.drop(["ID", "ZIP Code"],axis=1,inplace=True)

In [5]:
print("Duplicate : ", df.duplicated().sum())
print("Null : ", df.isnull().sum().sum())

Duplicate :  13
Null :  0


In [6]:
print(df["Experience"].unique())
df["Experience"] = abs(df["Experience"])

[ 1 19 15  9  8 13 27 24 10 39  5 23 32 41 30 14 18 21 28 31 11 16 20 35
  6 25  7 12 26 37 17  2 36 29  3 22 -1 34  0 38 40 33  4 -2 42 -3 43]


In [7]:
df.columns

Index(['Age', 'Experience', 'Income', 'Family', 'CCAvg', 'Education',
       'Mortgage', 'Personal Loan', 'Securities Account', 'CD Account',
       'Online', 'CreditCard'],
      dtype='object')

In [8]:
df = df[['Age', 'Experience', 'Income', 'Family', 'CCAvg','Education', 'Mortgage', 'Securities Account','CD Account', 'Online', 'CreditCard', 'Personal Loan']]

In [9]:
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

### Train Test Split

In [10]:
x,y = X,Y

In [11]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,random_state=69)

In [12]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [13]:
x_train.shape, y_train.shape

((3750, 11), (3750,))

# PyTorch 

In [14]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [15]:
BATCH_SIZE = 1

In [16]:
train_x = torch.from_numpy(x_train).to(torch.float32)
train_y = torch.from_numpy(y_train).to(torch.float32)

In [17]:
train_x.shape, train_y.shape

(torch.Size([3750, 11]), torch.Size([3750]))

In [18]:
data = TensorDataset(train_x,train_y)
data = DataLoader(data,batch_size=BATCH_SIZE,shuffle=True)

## Building Model

In [19]:
class Model(torch.nn.Module):
    
    def __init__(self):
        super(Model,self).__init__()
        
        self.layer1 = torch.nn.Linear(11,16)
        self.layer2 = torch.nn.Linear(16,1)
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.sigmoid(x)
        return x

In [20]:
model = Model()
# model.to(device)
print(model)

Model(
  (layer1): Linear(in_features=11, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


# Weight Optimization using Particle Swarm Optimization

In [21]:
from pyswarms.single import GlobalBestPSO
torch.manual_seed(420)
torch.set_grad_enabled(False)

model = Model()
param = np.concatenate([i.numpy().flatten() for i in model.parameters()])
shape = [i.numpy().shape for i in model.parameters()]
size = [i[0]*i[1] if len(i) == 2 else i[0] for i in shape]

print("Dim : ", len(param))
print("Layers Shape : ", shape)
print("Layers Size : ", size)

Dim :  209
Layers Shape :  [(16, 11), (16,), (1, 16), (1,)]
Layers Size :  [176, 16, 16, 1]


In [22]:
def objective_function(particle,shape=shape,size=size):
    accuracy = []
    output = []
    
    # Reshape the vector to weights and biases dimention
    for par in particle:
        param = list()
        cum_sum = 0
        for i in range(len(size)):
            array = par[cum_sum : cum_sum + size[i]]
            array = array.reshape(shape[i])
            cum_sum += size[i]
            param.append(array)
        param = np.array(param, dtype="object")
        output.append(param)
    
    for i in range(len(output)):
        # Copy Weights and Biases
        model = Model()
        for idx,wei in enumerate(model.parameters()):
            wei.data = (torch.tensor(output[i][idx])).to(torch.float32)
        
        # Calculate Accuracy
        y_pred = model(train_x)
        y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
        acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
        accuracy.append(1 - acc) # Optimization function aims to reduce the cost so (1 - accuracy)
        
    return accuracy

In [23]:
# Tunable Parameters

options = {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
dim = len(param)
x_max = 1.0 * np.ones(dim)
x_min = -1.0 * x_max
bounds = (x_min, x_max)

pso = GlobalBestPSO(n_particles=500, dimensions=209, options=options, bounds=bounds)

In [24]:
best_cost, best_parameters = pso.optimize(objective_function, iters=50)
print("Accuracy : ", 1 - best_cost)

2023-04-04 23:18:27,441 - pyswarms.single.global_best - INFO - Optimize for 50 iters with {'c1': 0.6, 'c2': 0.3, 'w': 0.1}
pyswarms.single.global_best: 100%|█████████████████████████████████████████████████████████████|50/50, best_cost=0.0483
2023-04-04 23:18:40,831 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.04826666666666668, best pos: [ 4.16909602e-01 -3.28622096e-01 -1.08419969e-01 -1.46689801e-01
  2.35199311e-01  6.06260628e-01 -1.95440160e-01 -4.02072627e-01
 -5.13891974e-01 -3.11041541e-01 -1.48321730e-01 -2.43779202e-01
 -5.70915090e-01  3.83220977e-01 -1.06306853e-01  3.29542994e-01
  5.91991436e-01  5.95688800e-01  1.80487536e-01  1.10245236e-01
 -4.04280621e-01  4.52045076e-01 -1.29524272e-02 -1.51995034e-01
  2.14406612e-01  9.07286363e-02 -8.65534458e-02 -9.54899804e-02
 -1.60334727e-01  8.12200241e-02  3.73922504e-01  1.49851243e-01
  1.32805298e-01  3.74202542e-01  3.05848384e-01  8.08967615e-03
 -2.12997794e-01 -3.72947095e-02 -1.0837357

Accuracy :  0.9517333333333333


In [25]:
# Converting the best vector to model

result = []
for par in [best_parameters]:
    param = list()
    cum_sum = 0
    for i in range(len(size)):
        array = par[cum_sum : cum_sum + size[i]]
        array = array.reshape(shape[i])
        cum_sum += size[i]
        param.append(array)
    param = np.array(param, dtype="object")
    result.append(param)

best_model = Model()
for idx,wei in enumerate(best_model.parameters()):
    wei.data = (torch.tensor(result[0][idx])).to(torch.float32)

# Calculate Accuracy
y_pred = best_model(train_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
acc = (y_pred == train_y).sum().float().item() / len(data.dataset)
print("Accuracy : ", acc)

Accuracy :  0.9517333333333333


## Testing 

In [26]:
test_x = torch.from_numpy(x_test).to(torch.float32)
test_y = torch.from_numpy(y_test).to(torch.float32)

In [27]:
test = TensorDataset(test_x,test_y)
test = DataLoader(test,batch_size=1)

### Classification Report

In [28]:
y_pred = best_model(test_x)
y_pred = torch.where(y_pred>=0.5, 1, 0).flatten()
print(classification_report(y_pred,test_y))

              precision    recall  f1-score   support

           0       0.99      0.96      0.98      1170
           1       0.60      0.86      0.71        80

    accuracy                           0.95      1250
   macro avg       0.80      0.91      0.84      1250
weighted avg       0.97      0.95      0.96      1250

